<a href="https://colab.research.google.com/github/vikrambj2019/basic/blob/master/criss_rigg_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet
from sklearn.ensemble import RandomForestRegressor
import xgboost as XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import logging
import os
from datetime import datetime




def load_and_prepare_data(file_path):
    """Load and prepare the data for forecasting"""


    # Load the data
    data = pd.read_csv(file_path)

    # Convert string columns to numeric
    for col in ['mmos', 'Med-Total', 'Rx']:
        if data[col].dtype == 'object':
            data[col] = data[col].str.replace(',', '').astype(float)

    # Create date features
    data['date'] = pd.to_datetime(data['YYYYMM'].astype(str), format='%Y%m')
    data['year'] = data['date'].dt.year
    data['month'] = data['date'].dt.month
    data['quarter'] = data['date'].dt.quarter

    # Split into training and validation
    train_data = data[data['YYYYMM'] < 202501]
    valid_data = data[data['YYYYMM'] >= 202501]





    # Calculate and log PMPM (Per Member Per Month) metrics
    data['med_pmpm'] = data['Med-Total'] / data['mmos']
    data['rx_pmpm'] = data['Rx'] / data['mmos']


    return data, train_data, valid_data


In [41]:
def explore_time_series(data):
    """Perform time series analysis and log insights"""
    print("Performing time series analysis")

    # Aggregate data by month
    monthly_data = data.groupby('YYYYMM').agg(
        total_mmos=('mmos', 'sum'),
        total_med=('Med-Total', 'sum'),
        total_rx=('Rx', 'sum')
    ).reset_index()

    # Create date index for time series
    monthly_data['date'] = pd.to_datetime(monthly_data['YYYYMM'].astype(str), format='%Y%m')
    monthly_data.set_index('date', inplace=True)

    # Calculate growth rates
    monthly_data['mmos_growth'] = monthly_data['total_mmos'].pct_change() * 100
    monthly_data['med_growth'] = monthly_data['total_med'].pct_change() * 100
    monthly_data['rx_growth'] = monthly_data['total_rx'].pct_change() * 100

    # Calculate PMPM at aggregate level
    monthly_data['med_pmpm'] = monthly_data['total_med'] / monthly_data['total_mmos']
    monthly_data['rx_pmpm'] = monthly_data['total_rx'] / monthly_data['total_mmos']

    # Extract seasonality
    try:
        # Try additive decomposition
        mmos_decomp = seasonal_decompose(monthly_data['total_mmos'].dropna(), model='additive', period=12)

        # Get highest/lowest seasonal months
        seasonal = pd.DataFrame(mmos_decomp.seasonal)
        monthly_avg = seasonal.groupby(seasonal.index.month).mean()
        highest_month = monthly_avg.idxmax()[0]
        lowest_month = monthly_avg.idxmin()[0]

        month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        print(f"Membership typically peaks in {month_names[highest_month-1]} and is lowest in {month_names[lowest_month-1]} based on historical patterns", "SEASONALITY")

        # Check for trend
        trend_start = mmos_decomp.trend.dropna().iloc[0]
        trend_end = mmos_decomp.trend.dropna().iloc[-1]
        trend_pct_change = ((trend_end / trend_start) - 1) * 100

        if trend_pct_change > 5:
            print(f"Strong upward trend detected in membership with {trend_pct_change:.1f}% increase over the observed period", "TREND")
        elif trend_pct_change < -5:
            print(f"Downward trend detected in membership with {trend_pct_change:.1f}% decrease over the observed period", "TREND")
        else:
            print(f"Membership shows a relatively stable trend with only {trend_pct_change:.1f}% change over the observed period", "TREND")

    except Exception as e:
        print(f"Could not perform seasonal decomposition: {e}")

    # Analyze PMPM trends
    med_pmpm_start = monthly_data['med_pmpm'].dropna().iloc[:3].mean()
    med_pmpm_end = monthly_data['med_pmpm'].dropna().iloc[-3:].mean()
    med_pmpm_change = ((med_pmpm_end / med_pmpm_start) - 1) * 100

    rx_pmpm_start = monthly_data['rx_pmpm'].dropna().iloc[:3].mean()
    rx_pmpm_end = monthly_data['rx_pmpm'].dropna().iloc[-3:].mean()
    rx_pmpm_change = ((rx_pmpm_end / rx_pmpm_start) - 1) * 100

    print(f"Medical PMPM has changed by {med_pmpm_change:.1f}% from ${med_pmpm_start:.2f} to ${med_pmpm_end:.2f}", "PMPM")
    print(f"Rx PMPM has changed by {rx_pmpm_change:.1f}% from ${rx_pmpm_start:.2f} to ${rx_pmpm_end:.2f}", "PMPM")

    # Analyze monthly volatility
    mmos_volatility = monthly_data['mmos_growth'].std()
    med_volatility = monthly_data['med_growth'].std()
    rx_volatility = monthly_data['rx_growth'].std()

    print(f"Monthly volatility: Membership {mmos_volatility:.1f}%, Medical claims {med_volatility:.1f}%, Rx claims {rx_volatility:.1f}%", "VOLATILITY")

    # Calculate correlation between the metrics
    corr_med_mmos = monthly_data['total_med'].corr(monthly_data['total_mmos'])
    corr_rx_mmos = monthly_data['total_rx'].corr(monthly_data['total_mmos'])

    print(f"Correlation between Medical claims and Membership: {corr_med_mmos:.2f}", "CORRELATION")
    print(f"Correlation between Rx claims and Membership: {corr_rx_mmos:.2f}", "CORRELATION")

    return monthly_data


In [42]:
def analyze_segments_and_states(data):
    """Analyze segments and states to find patterns and log insights"""
    print("Analyzing segments and states")

    # Segment analysis
    segment_analysis = data.groupby('Segment').agg(
        avg_mmos=('mmos', 'mean'),
        total_mmos=('mmos', 'sum'),
        med_pmpm=('Med-Total', lambda x: x.sum() / data.loc[x.index, 'mmos'].sum()),
        rx_pmpm=('Rx', lambda x: x.sum() / data.loc[x.index, 'mmos'].sum())
    ).sort_values('total_mmos', ascending=False)

    print("Segment analysis:")
    print(segment_analysis)

    for segment in segment_analysis.index:
        print(f"Segment {segment}: Avg membership {segment_analysis.loc[segment, 'avg_mmos']:.0f}, " +
                   f"Med PMPM ${segment_analysis.loc[segment, 'med_pmpm']:.2f}, " +
                   f"Rx PMPM ${segment_analysis.loc[segment, 'rx_pmpm']:.2f}", "SEGMENT")

    # Compare segments by PMPM
    highest_med_pmpm = segment_analysis['med_pmpm'].idxmax()
    lowest_med_pmpm = segment_analysis['med_pmpm'].idxmin()
    highest_rx_pmpm = segment_analysis['rx_pmpm'].idxmax()
    lowest_rx_pmpm = segment_analysis['rx_pmpm'].idxmin()

    print(f"Highest Medical PMPM in {highest_med_pmpm} (${segment_analysis.loc[highest_med_pmpm, 'med_pmpm']:.2f}), " +
               f"lowest in {lowest_med_pmpm} (${segment_analysis.loc[lowest_med_pmpm, 'med_pmpm']:.2f})", "PMPM_VARIATION")

    print(f"Highest Rx PMPM in {highest_rx_pmpm} (${segment_analysis.loc[highest_rx_pmpm, 'rx_pmpm']:.2f}), " +
               f"lowest in {lowest_rx_pmpm} (${segment_analysis.loc[lowest_rx_pmpm, 'rx_pmpm']:.2f})", "PMPM_VARIATION")

    # State analysis
    state_analysis = data.groupby('State').agg(
        avg_mmos=('mmos', 'mean'),
        total_mmos=('mmos', 'sum'),
        med_pmpm=('Med-Total', lambda x: x.sum() / data.loc[x.index, 'mmos'].sum()),
        rx_pmpm=('Rx', lambda x: x.sum() / data.loc[x.index, 'mmos'].sum())
    ).sort_values('total_mmos', ascending=False)

    print("State analysis:")
    print(state_analysis)

    top_5_states = state_analysis.head(5)
    for state in top_5_states.index:
        print(f"State {state}: Avg membership {top_5_states.loc[state, 'avg_mmos']:.0f}, " +
                   f"Med PMPM ${top_5_states.loc[state, 'med_pmpm']:.2f}, " +
                   f"Rx PMPM ${top_5_states.loc[state, 'rx_pmpm']:.2f}", "STATE")

    # Identify states with highest/lowest PMPMs
    highest_med_pmpm_state = state_analysis['med_pmpm'].idxmax()
    lowest_med_pmpm_state = state_analysis['med_pmpm'].idxmin()

    print(f"Highest Medical PMPM in state {highest_med_pmpm_state} (${state_analysis.loc[highest_med_pmpm_state, 'med_pmpm']:.2f}), " +
               f"lowest in {lowest_med_pmpm_state} (${state_analysis.loc[lowest_med_pmpm_state, 'med_pmpm']:.2f})", "STATE_PMPM")

    return segment_analysis, state_analysis


In [53]:
def train_and_forecast_sarima(data, target_col, group_cols=None):
    """
    Train SARIMA model and generate forecasts

    Parameters:
    -----------
    data : DataFrame
        Input data
    target_col : str
        Target column to forecast
    group_cols : list, optional
        Columns to group by. If None, forecast is done at the aggregate level.

    Returns:
    --------
    DataFrame
        Forecasts for 2025
    """
    print(f"Training SARIMA model for {target_col}")

    if group_cols is None:
        # Aggregate level forecast
        monthly_data = data.groupby('YYYYMM').agg({target_col: 'sum'}).reset_index()
        monthly_data['date'] = pd.to_datetime(monthly_data['YYYYMM'].astype(str), format='%Y%m')
        monthly_data.set_index('date', inplace=True)

        # Train data (pre-2025)
        train_data = monthly_data[monthly_data.index < pd.Timestamp('2025-01-01')]

        try:
            # Fit SARIMA model
            model = SARIMAX(train_data[target_col],
                           order=(1, 1, 1),
                           seasonal_order=(1, 1, 1, 12),
                           enforce_stationarity=False)
            results = model.fit(disp=False)

            # Forecast for 2025
            forecast_steps = 12
            forecast = results.get_forecast(steps=forecast_steps)
            forecast_mean = forecast.predicted_mean

            # Create forecast dataframe
            forecast_dates = pd.date_range(start=pd.Timestamp('2025-01-01'), periods=forecast_steps, freq='MS')
            forecast_df = pd.DataFrame({
                'date': forecast_dates,
                'YYYYMM': forecast_dates.strftime('%Y%m').astype(int),
                f'{target_col}_forecast': forecast_mean.values
            })

            print(f"SARIMA aggregate forecast for {target_col} completed successfully")
            print(f"SARIMA aggregate forecast for {target_col}: Mean for 2025 = {forecast_mean.mean():.2f}", "FORECAST")

            return forecast_df

        except Exception as e:
            #print(f"Error in SARIMA forecasting for {target_col}: {e}")
            return pd.DataFrame(columns=['date', 'YYYYMM', f'{target_col}_forecast'])

    else:
        # Group level forecasts
        forecasts = []

        # Get unique combinations of group columns
        groups = data[group_cols].drop_duplicates()

        print(f"Starting SARIMA forecasts for {len(groups)} groups")

        for _, group in groups.iterrows():
            group_filter = True
            for col in group_cols:
                group_filter = group_filter & (data[col] == group[col])

            group_data = data[group_filter].copy()

            # Skip if not enough data
            if len(group_data) < 12:
                #print(f"Skipping {group.to_dict()} due to insufficient data")
                continue

            # Aggregate by month
            monthly = group_data.groupby('YYYYMM').agg({target_col: 'sum'}).reset_index()
            monthly['date'] = pd.to_datetime(monthly['YYYYMM'].astype(str), format='%Y%m')
            monthly.set_index('date', inplace=True)

            # Train data (pre-2025)
            train_monthly = monthly[monthly.index < pd.Timestamp('2025-01-01')]

            try:
                # Fit SARIMA model with simplified parameters for stability
                model = SARIMAX(train_monthly[target_col],
                               order=(1, 1, 0),
                               seasonal_order=(0, 1, 1, 12),
                               enforce_stationarity=False)
                results = model.fit(disp=False)

                # Forecast for 2025
                forecast_steps = 12
                forecast = results.get_forecast(steps=forecast_steps)
                forecast_mean = forecast.predicted_mean

                # Create forecast dataframe
                forecast_dates = pd.date_range(start=pd.Timestamp('2025-01-01'), periods=forecast_steps, freq='MS')
                forecast_df = pd.DataFrame({
                    'date': forecast_dates,
                    'YYYYMM': forecast_dates.strftime('%Y%m').astype(int),
                    f'{target_col}_forecast': forecast_mean.values
                })

                # Add group columns
                for col in group_cols:
                    forecast_df[col] = group[col]

                forecasts.append(forecast_df)
                print(f"SARIMA forecast for {group.to_dict()} completed")

            except Exception as e:
                #print(f"Error in SARIMA forecasting for {group.to_dict()}: {e}")

              if not forecasts:
                return pd.DataFrame(columns=['date', 'YYYYMM', f'{target_col}_forecast'] + group_cols)

        # Combine all forecasts
        all_forecasts = pd.concat(forecasts, ignore_index=True)
        print(f"SARIMA forecasting completed for {len(forecasts)} out of {len(groups)} groups")

        return all_forecasts



In [57]:
#-----Prophet model------------------------------------------------------------------------------------
def train_and_forecast_prophet(data, target_col, group_cols=None):
    """
    Train Prophet model and generate forecasts

    Parameters:
    -----------
    data : DataFrame
        Input data
    target_col : str
        Target column to forecast
    group_cols : list, optional
        Columns to group by. If None, forecast is done at the aggregate level.

    Returns:
    --------
    DataFrame
        Forecasts for 2025
    """
    print(f"Training Prophet model for {target_col}")

    if group_cols is None:
        # Aggregate level forecast
        monthly_data = data.groupby('YYYYMM').agg({target_col: 'sum'}).reset_index()
        monthly_data['ds'] = pd.to_datetime(monthly_data['YYYYMM'].astype(str), format='%Y%m')
        monthly_data['y'] = monthly_data[target_col]

        # Train data (pre-2025)
        train_data = monthly_data[monthly_data['ds'] < pd.Timestamp('2025-01-01')]

        try:
            # Train Prophet model
            model = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
            model.fit(train_data[['ds', 'y']])

            # Create future dataframe for forecasting
            future = pd.DataFrame({'ds': pd.date_range(start='2025-01-01', periods=12, freq='MS')})

            # Make forecast
            forecast = model.predict(future)

            # Create forecast dataframe
            forecast_df = pd.DataFrame({
                'date': forecast['ds'],
                'YYYYMM': forecast['ds'].dt.strftime('%Y%m').astype(int),
                f'{target_col}_forecast': forecast['yhat']
            })

            print(f"Prophet aggregate forecast for {target_col} completed successfully")
            print(f"Prophet aggregate forecast for {target_col}: Mean for 2025 = {forecast['yhat'].mean():.2f}", "FORECAST")

            return forecast_df

        except Exception as e:
            #print(f"Error in Prophet forecasting for {target_col}: {e}")
            return pd.DataFrame(columns=['date', 'YYYYMM', f'{target_col}_forecast'])

    else:
        # Group level forecasts
        forecasts = []

        # Get unique combinations of group columns
        groups = data[group_cols].drop_duplicates()

        print(f"Starting Prophet forecasts for {len(groups)} groups")

        for _, group in groups.iterrows():
            group_filter = True
            for col in group_cols:
                group_filter = group_filter & (data[col] == group[col])

            group_data = data[group_filter].copy()

            # Skip if not enough data
            if len(group_data) < 12:
                #print(f"Skipping {group.to_dict()} due to insufficient data")
                continue

            # Aggregate by month
            monthly = group_data.groupby('YYYYMM').agg({target_col: 'sum'}).reset_index()
            monthly['ds'] = pd.to_datetime(monthly['YYYYMM'].astype(str), format='%Y%m')
            monthly['y'] = monthly[target_col]

            # Train data (pre-2025)
            train_monthly = monthly[monthly['ds'] < pd.Timestamp('2025-01-01')]

            try:
                # Train Prophet model
                model = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
                model.fit(train_monthly[['ds', 'y']])

                # Create future dataframe for forecasting
                future = pd.DataFrame({'ds': pd.date_range(start='2025-01-01', periods=12, freq='MS')})

                # Make forecast
                forecast = model.predict(future)

                # Create forecast dataframe
                forecast_df = pd.DataFrame({
                    'date': forecast['ds'],
                    'YYYYMM': forecast['ds'].dt.strftime('%Y%m').astype(int),
                    f'{target_col}_forecast': forecast['yhat']
                })

                # Add group columns
                for col in group_cols:
                    forecast_df[col] = group[col]

                forecasts.append(forecast_df)
                print(f"Prophet forecast for {group.to_dict()} completed")

            except Exception as e:
                #print(f"Error in Prophet forecasting for {group.to_dict()}: {e}")

              if not forecasts:
                return pd.DataFrame(columns=['date', 'YYYYMM', f'{target_col}_forecast'] + group_cols)

        # Combine all forecasts
        all_forecasts = pd.concat(forecasts, ignore_index=True)
        print(f"Prophet forecasting completed for {len(forecasts)} out of {len(groups)} groups")

        return all_forecasts



In [58]:
#-----ML model------------------------------------------------------------------------------------
def train_and_forecast_ml(data, target_col, group_cols=None):
    """
    Train ML models (Random Forest and XGBoost) and generate forecasts

    Parameters:
    -----------
    data : DataFrame
        Input data
    target_col : str
        Target column to forecast
    group_cols : list, optional
        Columns to group by. If None, forecast is done at the aggregate level.

    Returns:
    --------
    DataFrame
        Forecasts for 2025
    """
    print(f"Training ML models for {target_col}")

    # Create time features for 2025
    future_months = pd.DataFrame({
        'YYYYMM': pd.date_range(start='2025-01-01', periods=12, freq='MS').strftime('%Y%m').astype(int),
        'date': pd.date_range(start='2025-01-01', periods=12, freq='MS'),
        'year': 2025,
        'month': range(1, 13),
        'quarter': [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4]
    })

    if group_cols is None:
        # Prepare data for ML at aggregate level
        monthly_data = data.groupby('YYYYMM').agg({
            target_col: 'sum',
            'year': 'first',
            'month': 'first',
            'quarter': 'first'
        }).reset_index()

        # Create lag features
        for lag in [1, 2, 3, 6, 12]:
            monthly_data[f'{target_col}_lag_{lag}'] = monthly_data[target_col].shift(lag)

        # Create rolling window statistics
        for window in [3, 6, 12]:
            monthly_data[f'{target_col}_roll_mean_{window}'] = monthly_data[target_col].rolling(window=window).mean()

        # Train data (pre-2025)
        train_data = monthly_data[monthly_data['year'] < 2025].copy()
        train_data = train_data.dropna(subset=[col for col in train_data.columns if col.startswith(f'{target_col}_')])

        # Feature columns
        feature_cols = [col for col in train_data.columns if col.startswith(f'{target_col}_') or col in ['month', 'quarter']]

        # Check if we have enough data
        if len(train_data) < 24 or len(feature_cols) < 3:
            #print(f"Insufficient data for ML model training at aggregate level")
            return pd.DataFrame(columns=['date', 'YYYYMM', f'{target_col}_forecast_rf', f'{target_col}_forecast_xgb'])

        try:
            # Train Random Forest
            rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
            rf_model.fit(train_data[feature_cols], train_data[target_col])

            # Train XGBoost
            xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
            xgb_model.fit(train_data[feature_cols], train_data[target_col])

            # Prepare future data for prediction
            future_data = future_months.copy()

            # Add lag features based on the last available values
            last_values = monthly_data.sort_values('YYYYMM', ascending=False).iloc[0:12]

            for lag in [1, 2, 3, 6, 12]:
                if lag <= len(last_values):
                    future_data[f'{target_col}_lag_{lag}'] = last_values[target_col].iloc[lag-1]
                else:
                    future_data[f'{target_col}_lag_{lag}'] = last_values[target_col].iloc[0]

            # Add rolling mean features
            for window in [3, 6, 12]:
                future_data[f'{target_col}_roll_mean_{window}'] = last_values[target_col].iloc[0:min(window, len(last_values))].mean()

            # Make predictions
            future_data[f'{target_col}_forecast_rf'] = rf_model.predict(future_data[feature_cols])
            future_data[f'{target_col}_forecast_xgb'] = xgb_model.predict(future_data[feature_cols])

            # Create forecast dataframe
            forecast_df = future_data[['date', 'YYYYMM', f'{target_col}_forecast_rf', f'{target_col}_forecast_xgb']].copy()

            # Log ML model insights
            feature_importance_rf = pd.DataFrame({
                'feature': feature_cols,
                'importance': rf_model.feature_importances_
            }).sort_values('importance', ascending=False)

            top_features = feature_importance_rf.head(3)['feature'].tolist()
            print(f"Top 3 features for {target_col} Random Forest model: {', '.join(top_features)}", "ML_FEATURES")

            print(f"ML aggregate forecast for {target_col} completed successfully")
            print(f"ML aggregate forecast for {target_col}: RF Mean for 2025 = {forecast_df[f'{target_col}_forecast_rf'].mean():.2f}", "FORECAST")
            print(f"ML aggregate forecast for {target_col}: XGB Mean for 2025 = {forecast_df[f'{target_col}_forecast_xgb'].mean():.2f}", "FORECAST")

            return forecast_df

        except Exception as e:
            #print(f"Error in ML forecasting for {target_col}: {e}")
            return pd.DataFrame(columns=['date', 'YYYYMM', f'{target_col}_forecast_rf', f'{target_col}_forecast_xgb'])

    else:
        # Group level forecasts
        forecasts = []

        # Get unique combinations of group columns
        groups = data[group_cols].drop_duplicates()

        print(f"Starting ML forecasts for {len(groups)} groups")

        for _, group in groups.iterrows():
            group_filter = True
            for col in group_cols:
                group_filter = group_filter & (data[col] == group[col])

            group_data = data[group_filter].copy()

            # Skip if not enough data
            if len(group_data) < 24:
                #print(f"Skipping {group.to_dict()} due to insufficient data")
                continue

            # Aggregate by month
            monthly = group_data.groupby('YYYYMM').agg({
                target_col: 'sum',
                'year': 'first',
                'month': 'first',
                'quarter': 'first'
            }).reset_index()

            # Create lag features
            for lag in [1, 2, 3, 6, 12]:
                monthly[f'{target_col}_lag_{lag}'] = monthly[target_col].shift(lag)

            # Create rolling window statistics
            for window in [3, 6, 12]:
                monthly[f'{target_col}_roll_mean_{window}'] = monthly[target_col].rolling(window=window).mean()

            # Train data (pre-2025)
            train_monthly = monthly[monthly['year'] < 2025].copy()
            train_monthly = train_monthly.dropna(subset=[col for col in train_monthly.columns if col.startswith(f'{target_col}_')])

            # Feature columns
            feature_cols = [col for col in train_monthly.columns if col.startswith(f'{target_col}_') or col in ['month', 'quarter']]

            # Check if we have enough data
            if len(train_monthly) < 12 or len(feature_cols) < 3:
                #print(f"Insufficient data for ML model training for {group.to_dict()}")
                continue

            try:
                # Train Random Forest
                rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
                rf_model.fit(train_monthly[feature_cols], train_monthly[target_col])

                # Train XGBoost
                xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
                xgb_model.fit(train_monthly[feature_cols], train_monthly[target_col])

                # Prepare future data for prediction
                future_data = future_months.copy()

                # Add lag features based on the last available values
                last_values = monthly.sort_values('YYYYMM', ascending=False).iloc[0:12]

                for lag in [1, 2, 3, 6, 12]:
                    if lag <= len(last_values):
                        future_data[f'{target_col}_lag_{lag}'] = last_values[target_col].iloc[lag-1]
                    else:
                        future_data[f'{target_col}_lag_{lag}'] = last_values[target_col].iloc[0]

                # Add rolling mean features
                for window in [3, 6, 12]:
                    future_data[f'{target_col}_roll_mean_{window}'] = last_values[target_col].iloc[0:min(window, len(last_values))].mean()

                # Make predictions
                future_data[f'{target_col}_forecast_rf'] = rf_model.predict(future_data[feature_cols])
                future_data[f'{target_col}_forecast_xgb'] = xgb_model.predict(future_data[feature_cols])

                # Create forecast dataframe
                forecast_df = future_data[['date', 'YYYYMM', f'{target_col}_forecast_rf', f'{target_col}_forecast_xgb']].copy()

                # Add group columns
                for col in group_cols:
                    forecast_df[col] = group[col]

                forecasts.append(forecast_df)
                print(f"ML forecast for {group.to_dict()} completed")

            except Exception as e:
                #print(f"Error in ML forecasting for {group.to_dict()}: {e}")

              if not forecasts:
            #print(f"No successful ML forecasts for any group")
                return pd.DataFrame(columns=['date', 'YYYYMM', f'{target_col}_forecast_rf', f'{target_col}_forecast_xgb'] + group_cols)

        # Combine all forecasts
        all_forecasts = pd.concat(forecasts, ignore_index=True)
        print(f"ML forecasting completed for {len(forecasts)} out of {len(groups)} groups")

        return all_forecasts

In [59]:
def create_derived_forecasts(mmos_forecasts, data, group_cols=None):
    """
    Create derived forecasts for Med-Total and Rx based on MMOS forecasts

    Parameters:
    -----------
    mmos_forecasts : DataFrame
        DataFrame with MMOS forecasts
    data : DataFrame
        Original data
    group_cols : list, optional
        Columns to group by. If None, ratios are calculated at the aggregate level.

    Returns:
    --------
    DataFrame
        Forecasts with derived Med-Total and Rx
    """
    print("Creating derived forecasts from MMOS")

    if group_cols is None:
        # Calculate ratios at aggregate level
        monthly_data = data.groupby('YYYYMM').agg({
            'mmos': 'sum',
            'Med-Total': 'sum',
            'Rx': 'sum'
        }).reset_index()

        # Calculate ratios
        monthly_data['med_ratio'] = monthly_data['Med-Total'] / monthly_data['mmos']
        monthly_data['rx_ratio'] = monthly_data['Rx'] / monthly_data['mmos']

        # Use most recent ratios (last 6 months)
        recent_data = monthly_data.sort_values('YYYYMM', ascending=False).head(6)
        med_ratio = recent_data['med_ratio'].mean()
        rx_ratio = recent_data['rx_ratio'].mean()

        print(f"Using aggregate ratios for derived forecasts: Med={med_ratio:.2f}, Rx={rx_ratio:.2f}", "DERIVED_RATIOS")

        # Apply ratios to forecasts
        forecasts = mmos_forecasts.copy()

        # Apply to all MMOS forecast columns
        for col in mmos_forecasts.columns:
            if 'mmos_forecast' in col:
                base_name = col.replace('mmos_forecast', '')
                forecasts[f'Med-Total_forecast{base_name}'] = forecasts[col] * med_ratio
                forecasts[f'Rx_forecast{base_name}'] = forecasts[col] * rx_ratio

        return forecasts

    else:
        # Calculate ratios at group level
        ratio_data = data.groupby(group_cols).apply(
            lambda x: pd.Series({
                'med_ratio': x['Med-Total'].sum() / x['mmos'].sum() if x['mmos'].sum() > 0 else 0,
                'rx_ratio': x['Rx'].sum() / x['mmos'].sum() if x['mmos'].sum() > 0 else 0
            })
        ).reset_index()

        # Merge ratios with forecasts
        forecasts = mmos_forecasts.merge(ratio_data, on=group_cols, how='left')

        # Apply ratios to forecasts
        for col in mmos_forecasts.columns:
            if 'mmos_forecast' in col:
                base_name = col.replace('mmos_forecast', '')
                forecasts[f'Med-Total_forecast{base_name}'] = forecasts[col] * forecasts['med_ratio']
                forecasts[f'Rx_forecast{base_name}'] = forecasts[col] * forecasts['rx_ratio']

        # Log insights
        avg_med_ratio = ratio_data['med_ratio'].mean()
        avg_rx_ratio = ratio_data['rx_ratio'].mean()
        print(f"Average group-level ratios: Med={avg_med_ratio:.2f}, Rx={avg_rx_ratio:.2f}", "DERIVED_RATIOS")

        return forecasts


In [60]:
def create_ensemble_forecast(forecasts_dict, target_col, group_cols=None):
    """
    Create ensemble forecast from multiple forecast methods

    Parameters:
    -----------
    forecasts_dict : dict
        Dictionary of forecast DataFrames {method_name: forecast_df}
    target_col : str
        Target column to forecast
    group_cols : list, optional
        Columns to group by for merging forecasts

    Returns:
    --------
    DataFrame
        Ensemble forecast
    """
    print(f"Creating ensemble forecast for {target_col}")

    # Extract forecast methods
    methods = list(forecasts_dict.keys())
    print(f"Creating ensemble from methods: {methods}")

    if not methods:
        #print("No forecast methods provided for ensemble")
        return None

    merge_cols = ['YYYYMM']
    if group_cols is not None:
        merge_cols += group_cols

    # Start with first forecast
    ensemble = forecasts_dict[methods[0]].copy()

    # Merge other forecasts
    for method in methods[1:]:
        forecast_col = f"{target_col}_forecast_{method}"
        cols_to_use = merge_cols + [forecast_col]

        # Ensure column exists
        if forecast_col not in forecasts_dict[method].columns:
            forecast_cols = [col for col in forecasts_dict[method].columns if f'{target_col}_forecast' in col]
            if forecast_cols:
                # If multiple forecast columns exist (e.g., rf and xgb), use the first one
                forecasts_dict[method][forecast_col] = forecasts_dict[method][forecast_cols[0]]
            else:
                print(f"No forecast column for {target_col} in {method}")
                continue

        ensemble = ensemble.merge(
            forecasts_dict[method][cols_to_use],
            on=merge_cols,
            how='outer'
        )

    # Create ensemble forecast (mean of individual forecasts)
    forecast_cols = [col for col in ensemble.columns if f'{target_col}_forecast' in col]
    ensemble[f'{target_col}_forecast_ensemble'] = ensemble[forecast_cols].mean(axis=1)

    # Log ensemble insights
    print(f"Ensemble forecast for {target_col}: Mean for 2025 = {ensemble[f'{target_col}_forecast_ensemble'].mean():.2f}", "ENSEMBLE")

    return ensemble


In [61]:
def old_aggregate_forecasts(forecasts, target_col, grain='total', group_cols=None):
    """
    Aggregate forecasts to a specific grain

    Parameters:
    -----------
    forecasts : DataFrame
        Forecast DataFrame
    target_col : str
        Target column being forecasted
    grain : str
        Grain to aggregate to: 'total', 'state', 'segment'
    group_cols : list
        Group columns used for lowest grain forecasts

    Returns:
    --------
    DataFrame
        Aggregated forecasts
    """
    print(f"Aggregating {target_col} forecasts to {grain} level")

    if grain == 'total':
        agg_cols = ['YYYYMM']
    elif grain == 'state' and 'State' in forecasts.columns:
        agg_cols = ['State', 'YYYYMM']
    elif grain == 'segment' and 'Segment' in forecasts.columns:
        agg_cols = ['Segment', 'YYYYMM']
    else:
        print(f"Cannot aggregate to {grain} grain with available columns")
        return None

    # Get forecast columns
    forecast_cols = [col for col in forecasts.columns if f'{target_col}_forecast' in col]

    # Create aggregated dataframe
    agg_df = forecasts.groupby(agg_cols)[forecast_cols].sum().reset_index()

    return agg_df


In [62]:
def old_evaluate_forecasts(forecasts, actuals, target_col, group_cols=None):
    """
    Evaluate forecast accuracy against actual values

    Parameters:
    -----------
    forecasts : DataFrame
        Forecast DataFrame
    actuals : DataFrame
        Actual values DataFrame
    target_col : str
        Target column being forecasted
    group_cols : list, optional
        Columns to group by for merging

    Returns:
    --------
    dict
        Dictionary of evaluation metrics by forecast method
    """
    print(f"Evaluating {target_col} forecasts")

    merge_cols = ['YYYYMM']
    if group_cols is not None:
        merge_cols += group_cols

    # Get forecast columns
    forecast_cols = [col for col in forecasts.columns if f'{target_col}_forecast' in col]

    if not forecast_cols:
        print(f"No forecast columns found for {target_col}")
        return {}

    # Merge forecasts with actuals
    comparison = forecasts.merge(
        actuals[merge_cols + [target_col]],
        on=merge_cols,
        how='inner'
    )

    if len(comparison) == 0:
        print("No matching data for evaluation")
        return {}

    # Calculate metrics for each forecast method
    metrics = {}
    for col in forecast_cols:
        method = col.replace(f'{target_col}_forecast_', '')

        # Drop rows where either actual or forecast values are NaN
        valid_mask = ~(np.isnan(comparison[target_col]) | np.isnan(comparison[col]))
        valid_comparison = comparison[valid_mask]

        if len(valid_comparison) == 0:
            print(f"No valid data for method {method}")
            metrics[method] = {
                'MAE': np.nan,
                'RMSE': np.nan,
                'MAPE': np.nan
            }
            continue

        mae = mean_absolute_error(valid_comparison[target_col], valid_comparison[col])
        rmse = np.sqrt(mean_squared_error(valid_comparison[target_col], valid_comparison[col]))

        # Calculate MAPE (handle zero values)
        non_zero_mask = valid_comparison[target_col] != 0
        if non_zero_mask.sum() > 0:
            mape = np.mean(np.abs((valid_comparison.loc[non_zero_mask, target_col] - valid_comparison.loc[non_zero_mask, col]) / valid_comparison.loc[non_zero_mask, target_col])) * 100
        else:
            mape = np.nan

        metrics[method] = {
            'MAE': mae,
            'RMSE': rmse,
            'MAPE': mape
        }

        # Format the output to handle NaN values
        mae_str = f"{mae:.2f}" if not np.isnan(mae) else "NaN"
        rmse_str = f"{rmse:.2f}" if not np.isnan(rmse) else "NaN"
        mape_str = f"{mape:.2f}%" if not np.isnan(mape) else "NaN"

        print(f"Metrics for {method}: MAE={mae_str}, RMSE={rmse_str}, MAPE={mape_str}")

    # Determine best method, ignoring NaN values
    if metrics:
        valid_methods = {k: v for k, v in metrics.items() if not np.isnan(v['MAPE'])}
        if valid_methods:
            best_method = min(valid_methods.items(), key=lambda x: x[1]['MAPE'])[0]
            print(f"Best forecasting method for {target_col}: {best_method} with MAPE={metrics[best_method]['MAPE']:.2f}%", "EVALUATION")
        else:
            print(f"No valid methods for {target_col} evaluation")

    return metrics

In [63]:
def log_forecasts_insights(forecasts, target_col):
    """
    Log insights about the forecasts

    Parameters:
    -----------
    forecasts : DataFrame
        Forecast DataFrame with ensemble column
    target_col : str
        Target column being forecasted
    """
    print(f"Generating forecast insights for {target_col}")

    forecast_col = f'{target_col}_forecast_ensemble'

    if forecast_col not in forecasts.columns:
        print(f"No ensemble forecast column found for {target_col}")
        return

    # Monthly pattern
    monthly_pattern = forecasts.groupby('YYYYMM')[forecast_col].sum().reset_index()
    monthly_pattern['month'] = pd.to_datetime(monthly_pattern['YYYYMM'].astype(str), format='%Y%m').dt.month

    if len(monthly_pattern) >= 12:
        peak_month = monthly_pattern.iloc[monthly_pattern[forecast_col].argmax()]['month']
        low_month = monthly_pattern.iloc[monthly_pattern[forecast_col].argmin()]['month']

        month_names = ['January', 'February', 'March', 'April', 'May', 'June',
                      'July', 'August', 'September', 'October', 'November', 'December']

        print(f"2025 {target_col} forecast peaks in {month_names[peak_month-1]} and is lowest in {month_names[low_month-1]}", "FORECAST_PATTERN")

    # Year-over-year growth (if historical data is given)
    if 'year' in forecasts.columns:
        yearly_data = forecasts.groupby('year')[forecast_col].sum()
        if 2024 in yearly_data.index and 2025 in yearly_data.index:
            yoy_growth = (yearly_data[2025] / yearly_data[2024] - 1) * 100
            print(f"Year-over-year growth for {target_col} (2024-2025): {yoy_growth:.2f}%", "YOY_GROWTH")

    # Segment insights
    if 'Segment' in forecasts.columns:
        segment_forecasts = forecasts.groupby('Segment')[forecast_col].sum().sort_values(ascending=False)
        top_segment = segment_forecasts.index[0]
        bottom_segment = segment_forecasts.index[-1]

        print(f"Top segment for {target_col} in 2025: {top_segment} with {segment_forecasts[top_segment]:.2f}", "SEGMENT_INSIGHTS")
        print(f"Bottom segment for {target_col} in 2025: {bottom_segment} with {segment_forecasts[bottom_segment]:.2f}", "SEGMENT_INSIGHTS")

    # State insights
    if 'State' in forecasts.columns:
        state_forecasts = forecasts.groupby('State')[forecast_col].sum().sort_values(ascending=False)
        top_states = state_forecasts.head(3)

        print(f"Top 3 states for {target_col} in 2025: " +
                   ", ".join([f"{state} ({value:.2f})" for state, value in top_states.items()]), "STATE_INSIGHTS")



In [64]:
# Forecast MMOS
# Step 1: Load and prepare data
data, train_data, valid_data = load_and_prepare_data('/content/cr_v1.csv')

# Step 2: Explore time series patterns
monthly_data = explore_time_series(data)

# Step 3: Analyze segments and states
segment_analysis, state_analysis = analyze_segments_and_states(data)
# Step 4: Forecast at lowest grain (State × Segment × Month)
lowest_grain = ['State', 'Segment']
print("Forecasting MMOS at lowest grain")
sarima_mmos = train_and_forecast_sarima(train_data, 'mmos', lowest_grain)
prophet_mmos = train_and_forecast_prophet(train_data, 'mmos', lowest_grain)
ml_mmos = train_and_forecast_ml(train_data, 'mmos', lowest_grain)

# Rename forecast columns
sarima_mmos = sarima_mmos.rename(columns={'mmos_forecast': 'mmos_forecast_sarima'})
prophet_mmos = prophet_mmos.rename(columns={'mmos_forecast': 'mmos_forecast_prophet'})

# Combine forecasts for ensemble
mmos_forecasts = {
        'sarima': sarima_mmos,
        'prophet': prophet_mmos,
        'rf': ml_mmos[['YYYYMM', 'State', 'Segment', 'mmos_forecast_rf']],
        'xgb': ml_mmos[['YYYYMM', 'State', 'Segment', 'mmos_forecast_xgb']]
    }

# Create ensemble
ensemble_mmos = create_ensemble_forecast(mmos_forecasts, 'mmos', lowest_grain)

# Forecast Med-Total directly
print("Forecasting Med-Total at lowest grain")
sarima_med = train_and_forecast_sarima(train_data, 'Med-Total', lowest_grain)
prophet_med = train_and_forecast_prophet(train_data, 'Med-Total', lowest_grain)
ml_med = train_and_forecast_ml(train_data, 'Med-Total', lowest_grain)

# Rename forecast columns
sarima_med = sarima_med.rename(columns={'Med-Total_forecast': 'Med-Total_forecast_sarima'})
prophet_med = prophet_med.rename(columns={'Med-Total_forecast': 'Med-Total_forecast_prophet'})

# Combine forecasts for ensemble
med_forecasts = {
        'sarima': sarima_med,
        'prophet': prophet_med,
        'rf': ml_med[['YYYYMM', 'State', 'Segment', 'Med-Total_forecast_rf']],
        'xgb': ml_med[['YYYYMM', 'State', 'Segment', 'Med-Total_forecast_xgb']]
    }

# Create ensemble
ensemble_med = create_ensemble_forecast(med_forecasts, 'Med-Total', lowest_grain)

# Forecast Rx directly
print("Forecasting Rx at lowest grain")
sarima_rx = train_and_forecast_sarima(train_data, 'Rx', lowest_grain)
prophet_rx = train_and_forecast_prophet(train_data, 'Rx', lowest_grain)
ml_rx = train_and_forecast_ml(train_data, 'Rx', lowest_grain)

# Rename forecast columns
sarima_rx = sarima_rx.rename(columns={'Rx_forecast': 'Rx_forecast_sarima'})
prophet_rx = prophet_rx.rename(columns={'Rx_forecast': 'Rx_forecast_prophet'})

# Combine forecasts for ensemble
rx_forecasts = {
        'sarima': sarima_rx,
        'prophet': prophet_rx,
        'rf': ml_rx[['YYYYMM', 'State', 'Segment', 'Rx_forecast_rf']],
        'xgb': ml_rx[['YYYYMM', 'State', 'Segment', 'Rx_forecast_xgb']]
    }

# Create ensemble
ensemble_rx = create_ensemble_forecast(rx_forecasts, 'Rx', lowest_grain)

# Create derived forecasts from MMOS
derived_forecasts = create_derived_forecasts(ensemble_mmos, train_data, lowest_grain)

# Combine direct and derived forecasts
print("Combining direct and derived forecasts")

# Merge the ensemble forecasts
final_forecasts = ensemble_mmos.merge(
        ensemble_med[['YYYYMM', 'State', 'Segment', 'Med-Total_forecast_ensemble']],
        on=['YYYYMM', 'State', 'Segment'],
        how='outer'
    )
final_forecasts = final_forecasts.merge(
        ensemble_rx[['YYYYMM', 'State', 'Segment', 'Rx_forecast_ensemble']],
        on=['YYYYMM', 'State', 'Segment'],
        how='outer'
    )

# Add derived forecasts


# You need to specify the full column names from derived_forecasts
final_forecasts = final_forecasts.merge(
    derived_forecasts[['YYYYMM', 'State', 'Segment', 'Med-Total_forecast_ensemble', 'Rx_forecast_ensemble']],
    on=['YYYYMM', 'State', 'Segment'],
    how='outer'
)


# Final ensemble: combine direct and derived forecasts with weights
final_forecasts['Med-Total_final'] = 0.7 * final_forecasts['Med-Total_forecast_ensemble_x'] + 0.3 * final_forecasts['Med-Total_forecast_ensemble_y']
final_forecasts['Rx_final'] = 0.7 * final_forecasts['Rx_forecast_ensemble_x'] + 0.3 * final_forecasts['Rx_forecast_ensemble_y']

# Create and log PMPM metrics for final forecasts



# Create and log PMPM metrics for final forecasts
final_forecasts['Med-Total_PMPM'] = final_forecasts.apply(
    lambda row: float('nan') if row['mmos_forecast_ensemble'] == 0
    else row['Med-Total_final'] / row['mmos_forecast_ensemble'],
    axis=1
)

final_forecasts['Rx_PMPM'] = final_forecasts.apply(
    lambda row: float('nan') if row['mmos_forecast_ensemble'] == 0
    else row['Rx_final'] / row['mmos_forecast_ensemble'],
    axis=1
)


avg_med_pmpm = final_forecasts['Med-Total_PMPM'].mean()
avg_rx_pmpm = final_forecasts['Rx_PMPM'].mean()

print(f"Forecasted 2025 Medical PMPM: ${avg_med_pmpm:.2f}", "FORECAST_PMPM")
print(f"Forecasted 2025 Rx PMPM: ${avg_rx_pmpm:.2f}", "FORECAST_PMPM")

# Evaluate forecasts if validation data is available
#if len(valid_data) > 0:
#    mmos_metrics = evaluate_forecasts(ensemble_mmos, valid_data, 'mmos', lowest_grain)
#    med_metrics = evaluate_forecasts(ensemble_med, valid_data, 'Med-Total', lowest_grain)
#    rx_metrics = evaluate_forecasts(ensemble_rx, valid_data, 'Rx', lowest_grain)

# Aggregate forecasts to State level
#state_forecasts = aggregate_forecasts(final_forecasts, 'mmos', 'state', lowest_grain)
#print(state_forecasts, 'mmos')

# Aggregate forecasts to Segment level
#segment_forecasts = aggregate_forecasts(final_forecasts, 'mmos', 'segment', lowest_grain)
#print(segment_forecasts, 'mmos')

# Aggregate forecasts to total level
#total_forecasts = aggregate_forecasts(final_forecasts, 'mmos', 'total', lowest_grain)
#print(total_forecasts, 'mmos')

# Save results to CSV
output_dir = "/content/forecast_results"
os.makedirs(output_dir, exist_ok=True)

final_forecasts.to_csv(os.path.join(output_dir, 'lowest_grain_forecasts.csv'), index=False)
#state_forecasts.to_csv(os.path.join(output_dir, 'state_forecasts.csv'), index=False)
#segment_forecasts.to_csv(os.path.join(output_dir, 'segment_forecasts.csv'), index=False)
#total_forecasts.to_csv(os.path.join(output_dir, 'total_forecasts.csv'), index=False)



Performing time series analysis
Membership typically peaks in Feb and is lowest in Dec based on historical patterns SEASONALITY
Membership shows a relatively stable trend with only 3.4% change over the observed period TREND
Medical PMPM has changed by 19.8% from $359.93 to $431.23 PMPM
Rx PMPM has changed by 32.9% from $121.91 to $162.03 PMPM
Monthly volatility: Membership 0.5%, Medical claims 8.2%, Rx claims 8.5% VOLATILITY
Correlation between Medical claims and Membership: 0.79 CORRELATION
Correlation between Rx claims and Membership: 0.74 CORRELATION
Analyzing segments and states
Segment analysis:
              avg_mmos  total_mmos    med_pmpm     rx_pmpm
Segment                                                   
LGP_NRF  126675.992308  49403637.0  384.581246  138.729586
IND_ACA   54669.634615  25585389.0  367.477895  134.198647
SBG_ACA   65341.643590  25483241.0  430.096578  155.971863
SBG_MEW   15787.553846   6157146.0  283.408839  101.807787
SBG_LEG    8279.441026   3228982.0  36

<ipython-input-41-daf401b27765>:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  highest_month = monthly_avg.idxmax()[0]
<ipython-input-41-daf401b27765>:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lowest_month = monthly_avg.idxmin()[0]
<ipython-input-42-815e1f60dc38>:37: RuntimeWarning: invalid value encountered in scalar divide
  med_pmpm=('Med-Total', lambda x: x.sum() / data.loc[x.index, 'mmos'].sum()),
<ipython-input-42-815e1f60dc38>:38: RuntimeWarning: invalid value encountered in scalar divide
  rx_pmpm=('Rx', lambda x: x.sum() / data.loc[x.index, 'mmos'].sum())
/usr/local/lib/python3.11/dist-packages/st

SARIMA forecast for {'State': 'CO', 'Segment': 'IND_ACA'} completed
SARIMA forecast for {'State': 'CT', 'Segment': 'IND_ACA'} completed
SARIMA forecast for {'State': 'GA', 'Segment': 'IND_ACA'} completed
SARIMA forecast for {'State': 'IN', 'Segment': 'IND_ACA'} completed
SARIMA forecast for {'State': 'KY', 'Segment': 'IND_ACA'} completed
SARIMA forecast for {'State': 'ME', 'Segment': 'IND_ACA'} completed


/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/mlemodel.py:1235: RuntimeWarning: invalid value encountered in divide
  np.inner(score_obs, score_obs) /
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided,

SARIMA forecast for {'State': 'MO', 'Segment': 'IND_ACA'} completed
SARIMA forecast for {'State': 'NH', 'Segment': 'IND_ACA'} completed
SARIMA forecast for {'State': 'NV', 'Segment': 'IND_ACA'} completed
SARIMA forecast for {'State': 'NY', 'Segment': 'IND_ACA'} completed
SARIMA forecast for {'State': 'OH', 'Segment': 'IND_ACA'} completed
SARIMA forecast for {'State': 'VA', 'Segment': 'IND_ACA'} completed


...

Prophet forecast for {'State': 'VA', 'Segment': 'LGP_NRF'} completed


03:04:58 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Prophet forecast for {'State': 'WI', 'Segment': 'LGP_NRF'} completed
Prophet forecasting completed for 108 out of 108 groups
Training ML models for Rx
Starting ML forecasts for 108 groups
Creating ensemble forecast for Rx
Creating ensemble from methods: ['sarima', 'prophet', 'rf', 'xgb']
Ensemble forecast for Rx: Mean for 2025 = 7005146.72 ENSEMBLE
Creating derived forecasts from MMOS
Average group-level ratios: Med=384.88, Rx=126.83 DERIVED_RATIOS
Combining direct and derived forecasts


<ipython-input-59-84dfe45d726d>:54: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ratio_data = data.groupby(group_cols).apply(


ZeroDivisionError: float division by zero

In [65]:

# Create and log PMPM metrics for final forecasts
final_forecasts['Med-Total_PMPM'] = final_forecasts.apply(
    lambda row: float('nan') if row['mmos_forecast_ensemble'] == 0
    else row['Med-Total_final'] / row['mmos_forecast_ensemble'],
    axis=1
)

final_forecasts['Rx_PMPM'] = final_forecasts.apply(
    lambda row: float('nan') if row['mmos_forecast_ensemble'] == 0
    else row['Rx_final'] / row['mmos_forecast_ensemble'],
    axis=1
)


avg_med_pmpm = final_forecasts['Med-Total_PMPM'].mean()
avg_rx_pmpm = final_forecasts['Rx_PMPM'].mean()

print(f"Forecasted 2025 Medical PMPM: ${avg_med_pmpm:.2f}", "FORECAST_PMPM")
print(f"Forecasted 2025 Rx PMPM: ${avg_rx_pmpm:.2f}", "FORECAST_PMPM")

# Evaluate forecasts if validation data is available
#if len(valid_data) > 0:
#    mmos_metrics = evaluate_forecasts(ensemble_mmos, valid_data, 'mmos', lowest_grain)
#    med_metrics = evaluate_forecasts(ensemble_med, valid_data, 'Med-Total', lowest_grain)
#    rx_metrics = evaluate_forecasts(ensemble_rx, valid_data, 'Rx', lowest_grain)

# Aggregate forecasts to State level
#state_forecasts = aggregate_forecasts(final_forecasts, 'mmos', 'state', lowest_grain)
#print(state_forecasts, 'mmos')

# Aggregate forecasts to Segment level
#segment_forecasts = aggregate_forecasts(final_forecasts, 'mmos', 'segment', lowest_grain)
#print(segment_forecasts, 'mmos')

# Aggregate forecasts to total level
#total_forecasts = aggregate_forecasts(final_forecasts, 'mmos', 'total', lowest_grain)
#print(total_forecasts, 'mmos')

# Save results to CSV
output_dir = "/content/forecast_results"
os.makedirs(output_dir, exist_ok=True)

final_forecasts.to_csv(os.path.join(output_dir, 'lowest_grain_forecasts.csv'), index=False)
#state_forecasts.to_csv(os.path.join(output_dir, 'state_forecasts.csv'), index=False)
#segment_forecasts.to_csv(os.path.join(output_dir, 'segment_forecasts.csv'), index=False)
#total_forecasts.to_csv(os.path.join(output_dir, 'total_forecasts.csv'), index=False)



Forecasted 2025 Medical PMPM: $504.52 FORECAST_PMPM
Forecasted 2025 Rx PMPM: $189.79 FORECAST_PMPM


In [22]:
# Add derived forecasts
#final_forecasts = final_forecasts.merge(
#        derived_forecasts[['YYYYMM', 'State', 'Segment', 'Med-Total_forecast_', 'Rx_forecast_']],
#        on=['YYYYMM', 'State', 'Segment'],
#        how='outer'
#    )
derived_forecasts.columns

Index(['date', 'YYYYMM', 'mmos_forecast_sarima', 'State', 'Segment',
       'mmos_forecast_prophet', 'mmos_forecast_rf', 'mmos_forecast_xgb',
       'mmos_forecast_ensemble', 'med_ratio', 'rx_ratio',
       'Med-Total_forecast_sarima', 'Rx_forecast_sarima',
       'Med-Total_forecast_prophet', 'Rx_forecast_prophet',
       'Med-Total_forecast_rf', 'Rx_forecast_rf', 'Med-Total_forecast_xgb',
       'Rx_forecast_xgb', 'Med-Total_forecast_ensemble',
       'Rx_forecast_ensemble'],
      dtype='object')

In [23]:
final_forecasts.columns

Index(['date', 'YYYYMM', 'mmos_forecast_sarima', 'State', 'Segment',
       'mmos_forecast_prophet', 'mmos_forecast_rf', 'mmos_forecast_xgb',
       'mmos_forecast_ensemble', 'Med-Total_forecast_ensemble',
       'Rx_forecast_ensemble'],
      dtype='object')

In [24]:
# You need to specify the full column names from derived_forecasts
final_forecasts = final_forecasts.merge(
    derived_forecasts[['YYYYMM', 'State', 'Segment', 'Med-Total_forecast_ensemble', 'Rx_forecast_ensemble']],
    on=['YYYYMM', 'State', 'Segment'],
    how='outer'
)

In [31]:
final_forecasts.columns

Index(['date', 'YYYYMM', 'mmos_forecast_sarima', 'State', 'Segment',
       'mmos_forecast_prophet', 'mmos_forecast_rf', 'mmos_forecast_xgb',
       'mmos_forecast_ensemble', 'Med-Total_forecast_ensemble_x',
       'Rx_forecast_ensemble_x', 'Med-Total_forecast_ensemble_y',
       'Rx_forecast_ensemble_y', 'Med-Total_final', 'Rx_final'],
      dtype='object')

In [37]:

# Final ensemble: combine direct and derived forecasts with weights
#final_forecasts['Med-Total_final'] = 0.7 * final_forecasts['Med-Total_forecast_ensemble_x'] + 0.3 * final_forecasts['Med-Total_forecast_ensemble_y']
#final_forecasts['Rx_final'] = 0.7 * final_forecasts['Rx_forecast_ensemble_x'] + 0.3 * final_forecasts['Rx_forecast_ensemble_y']

# Create and log PMPM metrics for final forecasts
#final_forecasts['Med-Total_PMPM'] = final_forecasts['Med-Total_final'] / final_forecasts['mmos_forecast_ensemble']
#final_forecasts['Rx_PMPM'] = final_forecasts['Rx_final'] / final_forecasts['mmos_forecast_ensemble']

# Create and log PMPM metrics for final forecasts
final_forecasts['Med-Total_PMPM'] = final_forecasts.apply(
    lambda row: float('nan') if row['mmos_forecast_ensemble'] == 0
    else row['Med-Total_final'] / row['mmos_forecast_ensemble'],
    axis=1
)

final_forecasts['Rx_PMPM'] = final_forecasts.apply(
    lambda row: float('nan') if row['mmos_forecast_ensemble'] == 0
    else row['Rx_final'] / row['mmos_forecast_ensemble'],
    axis=1
)


avg_med_pmpm = final_forecasts['Med-Total_PMPM'].mean()
avg_rx_pmpm = final_forecasts['Rx_PMPM'].mean()

print(f"Forecasted 2025 Medical PMPM: ${avg_med_pmpm:.2f}", "FORECAST_PMPM")
print(f"Forecasted 2025 Rx PMPM: ${avg_rx_pmpm:.2f}", "FORECAST_PMPM")

# Evaluate forecasts if validation data is available
#if len(valid_data) > 0:
#    mmos_metrics = evaluate_forecasts(ensemble_mmos, valid_data, 'mmos', lowest_grain)
#    med_metrics = evaluate_forecasts(ensemble_med, valid_data, 'Med-Total', lowest_grain)
#    rx_metrics = evaluate_forecasts(ensemble_rx, valid_data, 'Rx', lowest_grain)

# Aggregate forecasts to State level
state_forecasts = aggregate_forecasts(final_forecasts, 'mmos', 'state', lowest_grain)
print(state_forecasts, 'mmos')

# Aggregate forecasts to Segment level
segment_forecasts = aggregate_forecasts(final_forecasts, 'mmos', 'segment', lowest_grain)
print(segment_forecasts, 'mmos')

# Aggregate forecasts to total level
total_forecasts = aggregate_forecasts(final_forecasts, 'mmos', 'total', lowest_grain)
print(total_forecasts, 'mmos')

# Save results to CSV
output_dir = "/content/forecast_results"
os.makedirs(output_dir, exist_ok=True)

final_forecasts.to_csv(os.path.join(output_dir, 'lowest_grain_forecasts.csv'), index=False)
state_forecasts.to_csv(os.path.join(output_dir, 'state_forecasts.csv'), index=False)
segment_forecasts.to_csv(os.path.join(output_dir, 'segment_forecasts.csv'), index=False)
total_forecasts.to_csv(os.path.join(output_dir, 'total_forecasts.csv'), index=False)

# Create summary CSV of monthly forecasts
monthly_summary = total_forecasts[['YYYYMM', 'mmos_forecast_ensemble', 'Med-Total_final', 'Rx_final']].copy()
monthly_summary = monthly_summary.rename(columns={
        'mmos_forecast_ensemble': 'Membership',
        'Med-Total_final': 'Medical_Claims',
        'Rx_final': 'Rx_Claims'
    })
monthly_summary.to_csv(os.path.join(output_dir, 'monthly_summary_2025.csv'), index=False)


Forecasted 2025 Medical PMPM: $504.52 FORECAST_PMPM
Forecasted 2025 Rx PMPM: $189.79 FORECAST_PMPM
Aggregating mmos forecasts to state level
      State  YYYYMM  mmos_forecast_sarima  mmos_forecast_prophet  \
0    CA_HMO  202501         537877.658034          548797.650189   
1    CA_HMO  202502         538887.946293          557842.724593   
2    CA_HMO  202503         536892.300329          548007.191568   
3    CA_HMO  202504         538088.760034          546955.583904   
4    CA_HMO  202505         539182.363375          546346.579645   
..      ...     ...                   ...                    ...   
211      WI  202508         198923.675642          188989.144997   
212      WI  202509         200926.275592          190262.245497   
213      WI  202510         202711.263447          191293.710004   
214      WI  202511         202938.794030          192565.710553   
215      WI  202512         201980.372285          191770.414982   

    mmos_forecast_rf mmos_forecast_xgb mmo

KeyError: "['Med-Total_final', 'Rx_final'] not in index"

In [38]:
total_forecasts.columns


Index(['YYYYMM', 'mmos_forecast_sarima', 'mmos_forecast_prophet',
       'mmos_forecast_rf', 'mmos_forecast_xgb', 'mmos_forecast_ensemble'],
      dtype='object')

In [39]:
final_forecasts.columns

Index(['date', 'YYYYMM', 'mmos_forecast_sarima', 'State', 'Segment',
       'mmos_forecast_prophet', 'mmos_forecast_rf', 'mmos_forecast_xgb',
       'mmos_forecast_ensemble', 'Med-Total_forecast_ensemble_x',
       'Rx_forecast_ensemble_x', 'Med-Total_forecast_ensemble_y',
       'Rx_forecast_ensemble_y', 'Med-Total_final', 'Rx_final',
       'Med-Total_PMPM', 'Rx_PMPM'],
      dtype='object')